<a href="https://colab.research.google.com/github/MorganLee0906/self_study/blob/main/20210606_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ptt文章查詢器
#-*-coding:utf-8 -*-
from bs4 import BeautifulSoup as bs    #爬蟲套件
from wordcloud import WordCloud       #文字雲套件
import requests 
import jieba                 #文字分析套件
import matplotlib.pyplot as plt       #顯示圖形用
from datetime import datetime as dtime  #獲取時間用

#time
current_time = dtime.today() #get current time
print(current_time)

#File IO
f = open('tmp.txt', 'w', encoding='UTF-8') #open tmp.txt
f.write('') #clear file
f.close()

#requests
s = requests.session()
board_name = input('輸入看板名稱:')
next_page_url = 'https://www.ptt.cc/bbs/'+board_name+'/index.html'

#search data
mm_min , dd_min = map(int, input('輸入日期(該日期以後的文章)(mm dd)').split(' '))
mm_max , dd_max = map(int, input('輸入日期(該日期以前的文章)(mm dd)').split(' '))
val_min = int(input('搜尋推文數高於多少(<0則搜噓文數)的文章:'))

date_flag = True #check date range
count = 0

while date_flag:
  count += 1
  res = s.get(next_page_url)

  soup = bs(res.text, 'html.parser')
  div_tags = soup.find_all('div', {'class': 'title'}) #爬當前頁面所有文章標題
  val_tags = soup.find_all('div', {'class': 'nrec'}) #爬當前頁面所有推薦值
  date_tags = soup.find_all('div', {'class': 'date'}) #爬當前頁面所有日期標籤
  btn = soup.select('div.btn-group > a')
  up_page_href = btn[3]['href']

  article_list = [] #list of article title
  pushval_list = [] #list of article value
  artinfo_list = [] #list of article date

  for val_tag, div_tag, date_tag in zip(val_tags, div_tags, date_tags):
    num_tag= val_tag.find('span')
    a_tag = div_tag.find('a')
    d_tag = date_tag.get_text()
    artinfo_list.append(d_tag)
    if num_tag is None:
      pushval_list.append(0)
    else:
      pushval_list.append(num_tag.text)
    if a_tag is not None:
      article_list.append(a_tag.text)
    else:
      article_list.append('本文章已被刪除')

  '''
  註：因BBS的文章列表順序為由下往上排序，因此在加到tmp.txt前須先反轉list
  '''
  article_list.reverse()
  pushval_list.reverse()
  artinfo_list.reverse()

  f = open('tmp.txt', 'a', encoding='UTF-8')
  ft = open('/content/drive/MyDrive/Colab Notebooks/python 爬蟲/'+str(mm_min)+str(dd_min)+'-'+str(mm_max)+str(dd_max)+'_'+board_name, 'a', encoding='UTF-8')
  
  for date, val, title in zip(artinfo_list, pushval_list, article_list):
    if title !='本文章已被刪除':
      appe = title[(title.find(']')+1):]
    m,d = map(int, date.split('/'))
    if m < mm_max or (m==mm_max and d<=dd_max):
      f.write(appe+'\n')
      ft.write(str(date+'\t'+str(val)+'\t'+title+'\n'))
    '''
    註：因部分看板的置底文日期過早，導致程式檢查日期是可能會因此誤判。因此在檢查日期時不會檢查BBS WEB版的第一頁文章的所有日期，
    但可能因為這樣，在爬較少文章的看板時出錯。由於本次專案主要爬的是Stock版，屬於前十大看板，因此較不容易發生問題。
    若日後要爬其他看板，可能需要再修正判斷方式。
    '''
    if (m<mm_min or (m == mm_min and d < dd_min)) and count!=1:#檢查日期是否超出範圍
      date_flag = False
      break
    if type(val) == str:
      if val == '爆':
        val = 100
      elif val[0] == 'X':
        if val == 'XX':
          val = int(-100)
        else: 
          val = int(val[1])*-10
      val = int(val)
      #print(val)
    if val_min>=0:
      if val>=val_min:
        if (val == 100):
          print(date, '爆', title, sep = '\t')
        else:
          print(date, val, title, sep = '\t')
    else:
      if val<=val_min:
        if (val == -100):
          print(date, 'XX', title, sep = '\t')
        else:
          print(date, ('X'+str(int(val/-10))), title, sep = '\t')
  f.close()
  ft.close()
  if (date_flag) == False:
    break
  next_page_url = 'https://www.ptt.cc' + up_page_href

with open('tmp.txt', 'r', encoding = 'UTF-8') as rfile:
  text = rfile.read()
jieba.set_dictionary('/content/drive/MyDrive/Colab Notebooks/python 爬蟲/dict.txt')
wlist = jieba.cut(text)
words = " ".join(wlist)
countword = dict()
wordlist = words.split(' ')
for word in wordlist:
  if word.isalpha():
    #print(word)
    if word in countword:
      countword[word] = countword.get(word)+1
    else:
      countword[word] = 1
fl = open('/content/drive/MyDrive/Colab Notebooks/python 爬蟲/'+str(mm_min)+str(dd_min)+'-'+str(mm_max)+str(dd_max)+'_'+board_name+'_list.csv', 'a', encoding='UTF-8')
for word in countword:
  print(word, countword[word])
  fl.write(str(word)+','+str(countword[word])+'\n')
fl.close()
wc = WordCloud(width = 1080, height = 1080, scale = 3, min_font_size=50, max_font_size=300, background_color='white', collocations=False, font_path='/content/drive/MyDrive/Colab Notebooks/python 爬蟲/SourceHanSansTW-Regular.otf').generate(words)
plt.imshow(wc)
plt.axis("off")
plt.show()